# Project by Andrea Sanchietti

This is the second homework for the machine learning course.

It requires to design a model for classification of images from a dataset made of at least 10 diffrerent classes.

The dataset that i decided to use is the EuroSAT Dataset: a collection of satellitar images based on Sentinel-2 satellite images covering 13 spectral bands and consisting out of 10 classes with in total 27,000 labeled and geo-referenced images.

The first thing to do is to import libraries:

In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import Callback
from scikitplot.metrics import plot_confusion_matrix, plot_roc
import config
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

if False:
    import wandb
    from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
    run = wandb.init(project="ML-final-project", entity="andr3us")

The next step is reading the dataset:

In [ ]:
# Birds
train_ds = tf.keras.utils.image_dataset_from_directory(
  config.training_dir,
  seed=123,
  image_size=(config.img_height, config.img_width),
  batch_size=config.batch_size,
  shuffle=True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  config.valid_dir,
  seed=123,
  image_size=(config.img_height, config.img_width),
  batch_size=config.batch_size,
  shuffle=True)



In [ ]:
# EuroSat
train_ds = tf.keras.utils.image_dataset_from_directory(
  config.training_dir_eurosat,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(config.img_height, config.img_width),
  batch_size=config.batch_size,
  shuffle=True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  config.training_dir_eurosat,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(config.img_height, config.img_width),
  batch_size=config.batch_size,
  shuffle=True)



In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")
    if i >= 8:
      print(images[i].shape)
      print(labels[i].shape)

In [ ]:
# data argumentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomContrast(factor=0.1),
  tf.keras.layers.RandomZoom(height_factor=0.1, width_factor=0.1)
])

train_ds.map(lambda x, y: (data_augmentation(x), y, config.classes))


In [ ]:
# Normalizing the data

normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), tf.one_hot(y, config.classes)))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), tf.one_hot(y, config.classes)))

In [ ]:
# only for autoencoder training
# padding_layer = tf.keras.layers.ZeroPadding2D(38)

train_ds = train_ds.map(lambda x, y: (x, x))
val_ds = val_ds.map(lambda x, y: (x, x))

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:

def resblock(x, filters, kernelsize):
    fx = tf.keras.layers.Conv2D(filters, kernelsize, activation='relu', padding='same')(x)
    fx = tf.keras.layers.BatchNormalization()(fx)
    fx = tf.keras.layers.Conv2D(filters, kernelsize, padding='same')(fx)
    out = tf.keras.layers.Add()([x,fx])
    out = tf.keras.layers.ReLU()(out)
    out = tf.keras.layers.BatchNormalization()(out)
    return out


def convblock2d(x, filters, kernelsize: int, num_convs: int, activation="relu", padding="same"):
    if num_convs <= 0:
        return x
    out = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernelsize, padding=padding)(x)
    for i in range(num_convs - 1):
        out = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernelsize, activation="relu", padding=padding)(out)
    return out


def convblock2dregularized(x, filters, kernelsize: int, num_convs: int, activation="relu", padding="same"):
    if num_convs <= 0:
        return x
    out = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernelsize, padding=padding, kernel_regularizer="l2")(x)
    out = tf.keras.layers.Dropout(0.2)(out)
    for i in range(num_convs - 1):
        out = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernelsize, activation="relu", padding=padding, kernel_regularizer="l2")(out)
        out = tf.keras.layers.Dropout(0.2)(out)
    return out

def convblock2dregularized2(x, filters, kernelsize: int, num_convs: int, activation="relu", padding="same"):
    if num_convs <= 0:
        return x
    out = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernelsize, padding=padding, kernel_regularizer="l2")(x)
    out = tf.keras.layers.BatchNormalization()(out)
    for i in range(num_convs - 1):
        out = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernelsize, activation=activation, padding=padding, kernel_regularizer="l2")(out)
    return out

In [ ]:
# V1
def encoder(inputs):
    hidden_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(inputs)
    hidden_layer = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(hidden_layer) # 180 -> 90
    hidden_layer = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu")(hidden_layer)
    bottleneck = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(hidden_layer) # 90 -> 45    
    return bottleneck


def decoder(bottleneck):
    # input 16
    hidden_layer = tf.keras.layers.Conv2D(name='Decoder', filters=16, kernel_size=3, padding="same", activation="relu")(bottleneck)
    hidden_layer = tf.keras.layers.UpSampling2D((2, 2))(hidden_layer) # 45 -> 90
    hidden_layer = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu")(hidden_layer)
    hidden_layer = tf.keras.layers.UpSampling2D((2, 2))(hidden_layer) # 90 -> 180
    hidden_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu")(hidden_layer)
    output = tf.keras.layers.Conv2D(name="output", filters=3, kernel_size=5, padding="same", activation="sigmoid")(hidden_layer)

    return output


def autoencoder_1():
    inputs = tf.keras.Input(shape=config.input_shape)
    enc = encoder(inputs)
    outputs = decoder(enc)
    return keras.Model(inputs=inputs, outputs=outputs, name="ConvAutoencoder_v1"), 1


In [ ]:
# V2
def decoder(bottleneck):
    x = tf.keras.layers.Convolution2D(8, 7, activation='relu', padding='same')(bottleneck)
    x = tf.keras.layers.UpSampling2D((2, 2))(x) # 45 -> 90
    x = tf.keras.layers.Convolution2D(16, 7, activation='relu', padding='same')(x) 

    x = tf.keras.layers.UpSampling2D((2, 2))(x) # 90 -> 180
    decoded = tf.keras.layers.Convolution2D(3, 5, activation='sigmoid', padding='same')(x)

    return decoded

def encoder(inputs):
    x = tf.keras.layers.Convolution2D(16, 7, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x) # 180 -> 90
    x = tf.keras.layers.Convolution2D(8, 7, activation='relu', padding='same')(x)
    bottleneck = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x) # 90 -> 45 

    return bottleneck

def autoencoder_2():
    inputs = tf.keras.Input(shape=config.input_shape)
    enc = encoder(inputs)
    outputs = decoder(enc)
    return keras.Model(inputs=inputs, outputs=outputs, name="ConvAutoencoder_v2"), 1


In [ ]:
def resnet():
    return tf.keras.applications.resnet50.ResNet50(input_shape=config.input_shape, classes=config.classes, classifier_activation="softmax", weights=None), 0

In [ ]:
def inception():
    # input_tensor = tf.keras.Input(shape=(config.img_width, config.img_height, 3))
    incep = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(config.img_width, config.img_height, 3),
    pooling=None,
    classes=config.classes,
    classifier_activation="softmax",
    )
    for layer in incep.layers:
        layer.trainable = False
    incep.summary()

    last_layer = incep.get_layer("mixed10")
    print("Last layer:", last_layer)

    last_layer = last_layer.output

    # build a custrom top
    x = tf.keras.layers.Flatten()(last_layer)

    x = tf.keras.layers.Dense(2048, activation="relu")(x)

    # Dropout regularization to prevent overfitting
    x = tf.keras.layers.Dropout(0.3)(x)

    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(x)

    return keras.Model(inputs=incep.input, outputs=output_layer, name="Inception"), 0

In [ ]:
def model_l_6():

    inputs = tf.keras.Input(shape=config.input_shape)
    hidden_layer = convblock2dregularized2(inputs, 16, 5, 2)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 32, 5, 2)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 64, 5, 3)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 128, 3, 3)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 256, 3, 3)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Flatten()(hidden_layer)
    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(hidden_layer)

    return keras.Model(inputs=inputs, outputs=output_layer, name="ConvResNet_v2"), 0

In [ ]:
def model_l_5():

    inputs = tf.keras.Input(shape=config.input_shape)
    hidden_layer = convblock2dregularized2(inputs, 32, 3, 2)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 64, 3, 2)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 128, 3, 3)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 256, 3, 3)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized2(hidden_layer, 512, 3, 3)
    hidden_layer = tf.keras.layers.BatchNormalization()(hidden_layer)
    hidden_layer = tf.keras.layers.Flatten()(hidden_layer)
    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(hidden_layer)

    return keras.Model(inputs=inputs, outputs=output_layer, name="ConvResNet_v2"), 0

In [ ]:
def model_l_4():

    inputs = tf.keras.Input(shape=config.input_shape)
    hidden_layer = convblock2dregularized(inputs, 64, 3, 2)
    hidden_layer = resblock(hidden_layer, 64, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized(hidden_layer, 128, 3, 2)
    hidden_layer = resblock(hidden_layer, 128, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized(hidden_layer, 256, 3, 2)
    hidden_layer = resblock(hidden_layer, 256, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized(hidden_layer, 512, 3, 2)
    hidden_layer = resblock(hidden_layer, 512, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2dregularized(hidden_layer, 1024, 3, 3)
    hidden_layer = resblock(hidden_layer, 1024, 3)
    hidden_layer = tf.keras.layers.Flatten()(hidden_layer)
    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(hidden_layer)

    return keras.Model(inputs=inputs, outputs=output_layer, name="ConvResNet_v2"), 0

In [ ]:
def model_l_3(): # Same as model 2

    inputs = tf.keras.Input(shape=config.input_shape)
    hidden_layer = convblock2d(inputs, 16, 3, 2)
    hidden_layer = resblock(hidden_layer, 16, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 32, 3, 2)
    hidden_layer = resblock(hidden_layer, 32, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 64, 3, 2)
    hidden_layer = resblock(hidden_layer, 64, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 128, 3, 2)
    hidden_layer = resblock(hidden_layer, 128, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 256, 3, 3)
    hidden_layer = resblock(hidden_layer, 256, 3)
    hidden_layer = tf.keras.layers.Flatten()(hidden_layer)
    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(hidden_layer)

    return keras.Model(inputs=inputs, outputs=output_layer, name="ConvResNet_v2"), 0

In [ ]:
def model_l_2():

    inputs = tf.keras.Input(shape=config.input_shape)
    hidden_layer = convblock2d(inputs, 64, 3, 2)
    hidden_layer = resblock(hidden_layer, 64, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 128, 3, 2)
    hidden_layer = resblock(hidden_layer, 128, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=126, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 256, 3, 2)
    hidden_layer = resblock(hidden_layer, 256, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 512, 3, 2)
    hidden_layer = resblock(hidden_layer, 512, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 1024, 3, 3)
    hidden_layer = resblock(hidden_layer, 1024, 3)
    hidden_layer = tf.keras.layers.Flatten()(hidden_layer)
    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(hidden_layer)

    return keras.Model(inputs=inputs, outputs=output_layer, name="ConvResNet_v2"), 0

In [ ]:
def model_l_1():
    filt = 32
    inputs = tf.keras.Input(shape=config.input_shape)
    hidden_layer = convblock2d(inputs, filt, 3, 3)
    hidden_layer = resblock(hidden_layer, filt, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=filt, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, filt * 2, 3, 3)
    hidden_layer = resblock(hidden_layer, filt * 2, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=filt * 2, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, filt * 4, 3, 3)
    hidden_layer = resblock(hidden_layer, filt * 4, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=filt * 4, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, filt * 8, 3, 3)
    hidden_layer = resblock(hidden_layer, filt * 8, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=filt * 8, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, filt * 16, 3, 3)
    hidden_layer = resblock(hidden_layer, filt * 16, 3)
    hidden_layer = tf.keras.layers.Flatten()(hidden_layer)
    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(hidden_layer)

    return keras.Model(inputs=inputs, outputs=output_layer, name="ConvResNet_v1"), 0

In [ ]:
def model_0():

    inputs = tf.keras.Input(shape=config.input_shape)
    hidden_layer = convblock2d(inputs, 64, 3, 2)
    hidden_layer = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 126, 3, 2)
    hidden_layer = tf.keras.layers.Conv2D(filters=126, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 256, 3, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 512, 3, 3)
    hidden_layer = tf.keras.layers.Conv2D(filters=512, kernel_size=3, padding="same", activation="relu", strides=2)(hidden_layer)
    hidden_layer = convblock2d(hidden_layer, 1024, 3, 3)
    hidden_layer = tf.keras.layers.Flatten()(hidden_layer)
    output_layer = tf.keras.layers.Dense(config.classes, activation="softmax")(hidden_layer)

    return keras.Model(inputs=inputs, outputs=output_layer, name="FirstNet"), 0

In [ ]:
model, mode = autoencoder_1()
print(mode)
model.summary()
tf.keras.utils.plot_model(model,to_file="model.png", show_shapes=True, show_layer_names=True)

In [ ]:
tf.config.list_physical_devices('GPU') # True/False

In [ ]:
# for autoencoders

class PredictOne(Callback):

    def on_epoch_end(self, epoch: int, logs= None) -> None:
        """Called at the end of an epoch."""
        for X, y in train_ds:    
            y_pred = self.model.predict(X)

            for pred, true in zip(y_pred, y):
                print(pred.shape)
                print(true.shape)
                fig, ax = plt.subplots(1, 2)
                ax[0].imshow(pred)
                ax[1].imshow(true)
                plt.show()
                print(true[100, 100])
                print(pred[100, 100])
                break
            break


In [ ]:
if mode == 0:
  
  model.compile(
    optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=1e-4), # todo
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalCrossentropy(), tf.keras.metrics.CategoricalAccuracy()])

  history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    batch_size=config.batch_size,
    callbacks=[WandbMetricsLogger(log_freq="batch"),
    WandbModelCheckpoint("model", save_best_only=True)]
  )
elif mode == 1:

  # Autoencoder
  model.compile(
    optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=1e-4),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanSquaredError()])
  

  history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=200,
    batch_size=config.batch_size,
    callbacks=[WandbMetricsLogger(log_freq="batch"),
    WandbModelCheckpoint("models_autoencoder", save_best_only=True), 
    PredictOne()]
  )
  


In [ ]:
# Download a checkpoint model from wandb
import wandb
run = wandb.init()
artifact = run.use_artifact('andr3us/ML-final-project/run_3bn4rasz_model:v17', type='model')
artifact_dir = artifact.download()


In [ ]:
# Load a model
model = tf.keras.models.load_model(artifact_dir)
model.compile(metrics=[tf.keras.metrics.CategoricalCrossentropy(), tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(top_k=1), tf.keras.metrics.Recall()])

In [ ]:
results = model.evaluate(val_ds, batch_size=128)
print(results)

In [ ]:
# Get images of autoencoder

for X, y in train_ds:    
    y_pred = model.predict(X)

    for pred, true in zip(y_pred, y):
        print(pred.shape)
        print(true.shape)
        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(pred)
        ax[1].imshow(true)
        plt.show()
        print(true[100, 100])
        print(pred[100, 100])
        break
    break
# test /media/andrea/NVMe/Dataset/Eurosat/2750/Residential/Residential_1721.jpg


# layer_name = 'relu_conv2'
# model2= Model(inputs=model1.input, outputs=model1.get_layer(layer_name).output)

In [ ]:
y_pred = []
y_true = []
for X, y in val_ds:
    p = model.predict(X)
    y_pred += [_ for _ in tf.argmax(p, axis=1)]
    y_true += [_ for _ in tf.argmax(y, axis=1)]
y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
print(y_pred.shape)
print(y_true.shape)

In [ ]:
for i in range(10):
    print(tf.argmax(y_pred_one_hot[i]), y_true[i], "prediction: ", y_pred_one_hot[i])
    print()


In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(300, 300)
ConfusionMatrixDisplay.from_predictions(y_true, y_pred, cmap="PuBu", ax = ax)
fig.savefig(fname="BigMatrix")

In [ ]:
print(history.history)

In [ ]:
### classification with encoder
# remember to reload dataset and normalize it without using the part that removes labels

inp = model.input

bottleneck = model.layers[5].output # take the bottleneck from autoencoder 1

enc = tf.keras.Model(inp, bottleneck)
enc.trainable = False
enc.summary()



In [ ]:
# add the output SVM

hidden_l = tf.keras.layers.Flatten()(bottleneck)
hidden_l = tf.keras.layers.Dense(64, activation="relu")(hidden_l)
out = tf.keras.layers.Dense(10, activation="sigmoid")(hidden_l)

classifier = tf.keras.Model(inp, out)

classifier.compile(loss='squared_hinge',
              optimizer='adadelta',
              metrics=['accuracy'])

classifier.fit(train_ds,
    validation_data=val_ds,
    epochs=20,
    batch_size=config.batch_size,
    callbacks=[WandbMetricsLogger(log_freq="batch"),
    WandbModelCheckpoint("model", save_best_only=True)])